<a href="https://colab.research.google.com/github/rvargas42/Value_Growth_Challenge_ETSFactory/blob/main/data/extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description
This notebook will take data from sources, filter it and create files to be used in model training

In [3]:
import requests as req
import pandas as pd
import os

#### Data Directories

In [4]:
#DIRECTORIES
data_dir = os.getcwd()
root_dir = os.path.dirname(data_dir)
#FILES
all_data = os.listdir(data_dir)
etf_list = os.path.join(data_dir,"fmp_data_etf_list.csv")
#READ DATA
etf_list = pd.read_csv(etf_list, index_col=False)
etf_list

,symbol,name,price,exchange,exchangeShortName,type
0,ULTR,IQ Ultra Short Duration ETF,47.96000,New York Stock Exchange Arca,AMEX,etf
1,RXL,ProShares Ultra Health Care,102.61300,New York Stock Exchange Arca,AMEX,etf
2,FMNY,First Trust New York High Income Municipal ETF,26.66290,New York Stock Exchange Arca,AMEX,etf
3,UIMP.DE,UBS (Lux) Fund Solutions – MSCI USA Socially R...,196.32000,Frankfurt Stock Exchange,XETRA,etf
4,DANC.TO,Desjardins Alt Long/Short Equity Market Neutra...,22.26000,Toronto Stock Exchange,TSX,etf
...,...,...,...,...,...,...
11965,TQSM.TO,TD Q U.S. Small-Mid-Cap Equity ETF,22.43000,Toronto Stock Exchange,TSX,etf
11966,SSIL.L,WisdomTree Silver 1x Daily Short,6.45625,London Stock Exchange,LSE,etf
11967,GURU,Global X Guru Index ETF,42.95410,New York Stock Exchange Arca,AMEX,etf
11968,HMJI.TO,BetaPro Marijuana Companies Inverse ETF,23.61000,Toronto Stock Exchange,TSX,etf


#### Data Reading and Cleaning

In [5]:
etfs_us = etf_list[etf_list["exchangeShortName"].str.contains("AMEX")]
etfs_us.dropna()


,symbol,name,price,exchange,exchangeShortName,type
0,ULTR,IQ Ultra Short Duration ETF,47.9600,New York Stock Exchange Arca,AMEX,etf
1,RXL,ProShares Ultra Health Care,102.6130,New York Stock Exchange Arca,AMEX,etf
2,FMNY,First Trust New York High Income Municipal ETF,26.6629,New York Stock Exchange Arca,AMEX,etf
6,IDRV,iShares Self-Driving EV and Tech ETF,30.3700,New York Stock Exchange Arca,AMEX,etf
7,SPYB,SPDR S&P 500 Buyback ETF,58.9059,New York Stock Exchange Arca,AMEX,etf
...,...,...,...,...,...,...
11951,NACP,Impact Shares NAACP Minority Empowerment ETF,38.9000,New York Stock Exchange Arca,AMEX,etf
11957,BECO,BlackRock Future Climate and Sustainable Econo...,21.6881,New York Stock Exchange Arca,AMEX,etf
11958,XC,WisdomTree Emerging Markets ex-China Fund,31.0599,New York Stock Exchange Arca,AMEX,etf
11962,IVOL,Quadratic Interest Rate Volatility and Inflati...,18.3200,New York Stock Exchange Arca,AMEX,etf


In [57]:
etfs_us_growth = etfs_us[etfs_us["name"].str.contains("Growth", na=False)]
etfs_us_value = etfs_us[etfs_us["name"].str.contains("Value",na=False)]
etfs_growth_value = {
    "Growth": {i : {"description":"","price_history":[], "sector_exposure":[]} for i in etfs_us_growth.symbol}, 
    "Value": {i : {"description":"","price_history":[], "sector_exposure":[]} for i in etfs_us_value.symbol}
}
print(f"-> {len(etfs_growth_value["Growth"].keys())} US Growth ETFs")
print(f"-> {len(etfs_growth_value["Value"].keys())} US Value ETFs")
etfs_us_value

-> 94 US Growth ETFs
-> 102 US Value ETFs


,symbol,name,price,exchange,exchangeShortName,type
24,RZV,Invesco S&P SmallCap 600 Pure Value ETF,102.2810,New York Stock Exchange Arca,AMEX,etf
57,IWN,iShares Russell 2000 Value ETF,151.8200,New York Stock Exchange Arca,AMEX,etf
60,IVLU,iShares Edge MSCI Intl Value Factor ETF,28.7600,New York Stock Exchange Arca,AMEX,etf
83,JVAL,JPMorgan U.S. Value Factor ETF,40.5600,New York Stock Exchange Arca,AMEX,etf
140,JPSV,Jpmorgan Active Small Cap Value ETF,54.0500,New York Stock Exchange Arca,AMEX,etf
...,...,...,...,...,...,...
11363,IVE,iShares S&P 500 Value ETF,183.3000,New York Stock Exchange Arca,AMEX,etf
11397,GVLU,Gotham 1000 Value ETF,23.1700,New York Stock Exchange Arca,AMEX,etf
11497,IHYV,Invesco Corporate Income Value ETF,24.8350,New York Stock Exchange Arca,AMEX,etf
11591,VOOV,Vanguard S&P 500 Value Index Fund,177.1700,New York Stock Exchange Arca,AMEX,etf


#### Populate ETF Dictionary with relevant data

In [58]:
#main features we will request to apis
main_features = list(etfs_growth_value["Growth"]['AILG'].keys())
main_features

['description', 'price_history', 'sector_exposure']

In [61]:
import yfinance as yf
import random, time

for i in list(etfs_growth_value["Growth"].keys()):
    history = yf.Ticker(i).history(period="max")
    description = yf.Ticker(i).info
    etfs_growth_value["Growth"][i]["description"] = description['longBusinessSummary']) if "longBusinessSummary" in description.keys() else ""
    etfs_growth_value["Growth"][i]["price_history"] = history.to_json(orient='split', date_format='iso')

for i in list(etfs_growth_value["Value"].keys()):
    history = yf.Ticker(i).history(period="max")
    description = yf.Ticker(i).info
    etfs_growth_value["Value"][i]["description"] = description['longBusinessSummary'] if "longBusinessSummary" in description.keys() else ""
    etfs_growth_value["Value"][i]["price_history"] = history.to_json(orient='split', date_format='iso')

In [62]:
etfs_growth_value["Growth"]["VUG"]

{'description': 'The fund employs an indexing investment approach designed to track the performance of the index, a broadly diversified index predominantly made up of growth stocks of large U.S. companies. The advisor attempts to replicate the target index by investing all, or substantially all, of its assets in the stocks that make up the index, holding each stock in approximately the same proportion as its weighting in the index.',
 'price_history': '{"columns":["Open","High","Low","Close","Volume","Dividends","Stock Splits","Capital Gains"],"index":["2004-01-30T05:00:00.000Z","2004-02-02T05:00:00.000Z","2004-02-03T05:00:00.000Z","2004-02-04T05:00:00.000Z","2004-02-05T05:00:00.000Z","2004-02-06T05:00:00.000Z","2004-02-09T05:00:00.000Z","2004-02-10T05:00:00.000Z","2004-02-11T05:00:00.000Z","2004-02-12T05:00:00.000Z","2004-02-13T05:00:00.000Z","2004-02-17T05:00:00.000Z","2004-02-18T05:00:00.000Z","2004-02-19T05:00:00.000Z","2004-02-20T05:00:00.000Z","2004-02-23T05:00:00.000Z","2004-02-

In [67]:
import json

with open(os.path.join(data_dir,"etf_data","etfs_us_value_growth.json"),'w') as f:
    json.dump(etfs_growth_value,f,indent=4)